In [6]:
from MyLib import load_celebA
from MyLib import VAE
import torch
from torch import optim
import torch.nn as nn
from torch.nn import functional as F
from torchvision.utils import save_image
from torch.autograd import Variable
import pdb

In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        
        
def loss_function(recon_x,x,mu,logvar):
    BCE = F.binary_cross_entropy(recon_x,x.view(-1,4096))
    KLD = -.5 * torch.sum(1+logvar-mu.pow(2)-logvar.exp())
    KLD /= batch_size*4096
    #pdb.set_trace()
    KLD = KLD 
    return BCE+KLD, BCE, KLD


bce_loss = nn.MSELoss()

In [8]:
def train(epoch):
    
    cell_train_loss = 0    
    linear_model.train()
    for batch_idx, data in enumerate(cell_train):
        image = Variable(data['image']).float().view(-1,1,64,64).cuda()
        cell_acti = Variable(data['cell']).float().cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar,z = model(image)
        Linear_cell = linear_model(z)
        #pdb.set_trace()
        loss = bce_loss(Linear_cell,cell_acti)
        loss.backward()
        cell_train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % 10 == 0:
            #pdb.set_trace()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(cell_train.dataset),
                100. * batch_idx / len(cell_train),
                loss.data[0] / len(data)
            ))
    
    
    model.train()
    #Linear_model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        #data = data.view(-1,1,64,64)
        data = Variable(data['image']).float().view(-1,1,64,64)
        
        
        data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar,z = model(data)
        loss,bce,kld = loss_function(recon_batch, data, mu, logvar)
        #pdb.set_trace()
        #loss.backward()
        
            
        loss.backward()
        #bce.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % 10 == 0:
            #pdb.set_trace()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)
            ))
            print ('kld=',kld.data[0]/len(data))
        '''
        if batch_idx % 40000 == 0:    
            n = min(data.size(0), 8)
            comparison = torch.cat([data[:n],recon_batch.view(args.batch_size, 1, 64, 64)[:n]])
            save_image(comparison.data.cpu(),'results/reconstruction_'+'beta_'+str(batch_idx)+'perfomance.png')
        '''

        
    #pdb.set_trace()   
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / (len(train_loader)*batch_size)))
    
    print('====> Epoch: {} Average cell loss: {:.4f}'.format(
          epoch, cell_train_loss / (len(cell_train)*batch_size)))

def test(epoch):
    model.eval()
    test_loss = 0
    for i, data in enumerate(test_loader):
        #data = data.view(-1,1,64,64)
        
        data = Variable(data['image'], volatile=True).float().view(-1,1,64,64)
        
        data = data.cuda()
        recon_batch, mu, logvar,z = model(data)
        test_lo,bce,kld = loss_function(recon_batch, data, mu, logvar)
        test_loss += test_lo
        #pdb.set_trace()
        
        if i == 0:
          
            n = min(data.size(0), 8)
          
            comparison = torch.cat([data[:n],
                                  recon_batch.view(batch_size, 1, 64, 64)[:n]])
            save_image(comparison.data.cpu(),
                     'results/reconstruction_' + str(epoch)  +'.png', nrow=n)
        #print ('processed batch '+str(i))
        
        linear_model.eval()
        cell_test_loss = 0
        for batch_idx, data in enumerate(cell_test):
            image = Variable(data['image']).float().view(-1,1,64,64).cuda()
            cell_acti = Variable(data['cell']).float().cuda()
            #optimizer.zero_grad()
            recon_batch, mu, logvar,z = model(image)
            Linear_cell = linear_model(z)
            loss = bce_loss(Linear_cell,cell_acti)
            #loss.backward()
            cell_test_loss += loss.data[0]
            #optimizer.step()
            '''
            if batch_idx % 10 == 0:
            #pdb.set_trace()
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(cell_train.dataset),
                    100. * batch_idx / len(cell_train),
                    loss.data[0] / len(data)
                )) 
             '''   
    test_loss /= len(test_loader)*batch_size
    cell_test_loss /= len(cell_test)*batch_size
    print('====> Test set loss =', test_loss)
    print('====> Cell Test set loss =', cell_test_loss)

In [9]:

batch_size = 128

_,train_loader,test_loader = load_celebA.celeba_dataset()

_,cell_train, cell_test = load_celebA.facial_dataset()
model = VAE.DC_VAE()

linear_model = VAE.Linear_model()

model.apply(weights_init)
linear_model.apply(weights_init)
model.cuda()
linear_model.cuda()
params = list(linear_model.parameters()) + list(model.parameters())
optimizer = optim.Adam(params, lr=2e-4)

In [ ]:
for epoch in range(1, 101):
    train(epoch)
    test(epoch)
    sample = Variable(torch.randn(64,32))
    
    sample = sample.cuda()
    sample = model.decoder(sample).cpu()
    save_image(sample.data.view(64, 1, 64, 64),
               'recon_result/sample_' + str(epoch) +'beta' + '.png')

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Train Epoch: 1 [0/2162 (0%)]	Loss: 76.234161
Train Epoch: 1 [20/2162 (71%)]	Loss: 72.722473
Train Epoch: 1 [0/202599 (0%)]	Loss: 0.006389
kld= 0.00018484411702957004
Train Epoch: 1 [1280/202599 (1%)]	Loss: 0.005820
kld= 0.00026934617199003696
Train Epoch: 1 [2560/202599 (2%)]	Loss: 0.005660
kld= 0.0002064094878733158
Train Epoch: 1 [3840/202599 (2%)]	Loss: 0.005484
kld= 0.00011426545825088397
Train Epoch: 1 [5120/202599 (3%)]	Loss: 0.005330
kld= 8.637474820716307e-05
Train Epoch: 1 [6400/202599 (4%)]	Loss: 0.005176
kld= 6.115913856774569e-05
Train Epoch: 1 [7680/202599 (5%)]	Loss: 0.005022
kld= 5.8214325690642e-05
Train Epoch: 1 [8960/202599 (6%)]	Loss: 0.005037
kld= 4.7095298214117065e-05
Train Epoch: 1 [10240/202599 (6%)]	Loss: 0.005041
kld= 4.68558537249919e-05
Train Epoch: 1 [11520/202599 (7%)]	Loss: 0.004901
kld= 4.3020099838031456e-05
Train Epoch: 1 [12800/202599 (8%)]	Loss: 0.004918
kld= 3.943354022339918e-05
Train Epoch: 1 [14080/202599 (9%)]	Loss: 0.004763
kld= 4.8452162445755

Train Epoch: 1 [133120/202599 (82%)]	Loss: 0.004379
kld= 8.736978634260595e-05
Train Epoch: 1 [134400/202599 (83%)]	Loss: 0.004469
kld= 8.427399006905034e-05
Train Epoch: 1 [135680/202599 (84%)]	Loss: 0.004429
kld= 8.711586269782856e-05
Train Epoch: 1 [136960/202599 (84%)]	Loss: 0.004422
kld= 8.423057442996651e-05
Train Epoch: 1 [138240/202599 (85%)]	Loss: 0.004383
kld= 8.777755283517763e-05
Train Epoch: 1 [139520/202599 (86%)]	Loss: 0.004349
kld= 8.853895997162908e-05
Train Epoch: 1 [140800/202599 (87%)]	Loss: 0.004348
kld= 8.356025500688702e-05
Train Epoch: 1 [142080/202599 (88%)]	Loss: 0.004411
kld= 8.452444308204576e-05
Train Epoch: 1 [143360/202599 (88%)]	Loss: 0.004484
kld= 8.902711851987988e-05
Train Epoch: 1 [144640/202599 (89%)]	Loss: 0.004385
kld= 8.701929618837312e-05
Train Epoch: 1 [145920/202599 (90%)]	Loss: 0.004382
kld= 8.643874753033742e-05
Train Epoch: 1 [147200/202599 (91%)]	Loss: 0.004378
kld= 8.528180478606373e-05
Train Epoch: 1 [148480/202599 (92%)]	Loss: 0.004466


Train Epoch: 2 [101120/202599 (62%)]	Loss: 0.004529
kld= 8.310261182487011e-05
Train Epoch: 2 [102400/202599 (63%)]	Loss: 0.004490
kld= 8.614925172878429e-05
Train Epoch: 2 [103680/202599 (64%)]	Loss: 0.004436
kld= 8.437973883701488e-05
Train Epoch: 2 [104960/202599 (65%)]	Loss: 0.004508
kld= 8.46823095344007e-05
Train Epoch: 2 [106240/202599 (66%)]	Loss: 0.004518
kld= 8.384021202800795e-05
Train Epoch: 2 [107520/202599 (66%)]	Loss: 0.004409
kld= 9.124707867158577e-05
Train Epoch: 2 [108800/202599 (67%)]	Loss: 0.004524
kld= 8.242432522820309e-05
Train Epoch: 2 [110080/202599 (68%)]	Loss: 0.004458
kld= 8.465275459457189e-05
Train Epoch: 2 [111360/202599 (69%)]	Loss: 0.004512
kld= 8.098484249785542e-05
Train Epoch: 2 [112640/202599 (69%)]	Loss: 0.004584
kld= 8.266781514976174e-05
Train Epoch: 2 [113920/202599 (70%)]	Loss: 0.004546
kld= 8.276032167486846e-05
Train Epoch: 2 [115200/202599 (71%)]	Loss: 0.004468
kld= 8.276877633761615e-05
Train Epoch: 2 [116480/202599 (72%)]	Loss: 0.004491
k